In [1]:
import numpy as np
from ase import io
from ralign import options, routines

In [2]:
atoms0 = io.read('r005/100cobalt_j5.xyz', index=0)
atoms1 = io.read('r005/100cobalt_j5.xyz', index=1)

labels0 = np.empty((len(atoms0), 32), dtype='c')
for i, symbol in enumerate(atoms0.get_chemical_symbols()):
    labels0[i] = symbol + (32 - len(symbol))*' '
    
labels1 = np.empty((len(atoms1), 32), dtype='c')
for i, symbol in enumerate(atoms1.get_chemical_symbols()):
    labels1[i] = symbol + (32 - len(symbol))*' '

coords0 = np.transpose(atoms0.get_positions())
coords1 = np.transpose(atoms1.get_positions())

In [3]:
options.remap = True
options.biased = True
options.testing = True
options.matching = True
options.iterative = True
options.maxmatch = 10
options.scale = 1000.0
options.tolerance = 0.17
options.weighter = 'none                            '

routines.superpose(labels0, labels1, coords0, coords1, 10)

(2,
 array([-3.3e-07,  5.0e-08,  3.0e-07]),
 array([-3.25900e-05,  1.77807e-03, -6.13900e-03]),
 array([[ 35,  96,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 68,  68,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 27,  45,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 31,  72,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 24,  62,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 70,  16,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 25,  25,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 75,  75,   0,   0,   0,   0,   0,   0,   0,   0],
        [  4,  74,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 41,  26,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 94, 100,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 71,  71,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 84,  61,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 47,  47,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 83,  89,   0,   0,   0,   0,   0,   0,   0,   0],
    